In [26]:
import tensorflow as tf
import urllib.request
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.preprocessing import image
import os

In [12]:
url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip'
file_name = 'rps.zip'
training_dir = './tmp/rps/'
urllib.request.urlretrieve(url, file_name)

('rps.zip', <http.client.HTTPMessage at 0x7f177d1b1880>)

In [19]:
url ='https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip'
file_name = 'rps-test-set.zip'
validation_dir = './tmp/rps-test-set/'
urllib.request.urlretrieve(url, file_name)

('rps-test-set.zip', <http.client.HTTPMessage at 0x7f17586ab250>)

In [20]:
zip_ref = zipfile.ZipFile(file_name, 'r')
zip_ref.extractall('./tmp/')
zip_ref.close()

In [10]:
zip_ref = zipfile.ZipFile(file_name, 'r')
zip_ref.extractall('./tmp/')
zip_ref.close()

In [21]:
validation_datagen = ImageDataGenerator(rescale=1./255.)
validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size=(150, 150), class_mode='categorical')

Found 372 images belonging to 3 classes.


In [16]:
training_datagen = ImageDataGenerator(rescale=1./255., rotation_range=40, width_shift_range=0.2,
                                      height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True,
                                     fill_mode='nearest')
training_generator = training_datagen.flow_from_directory(training_dir, target_size=(150, 150), class_mode='categorical')

Found 2520 images belonging to 3 classes.


In [17]:
model = tf.keras.models.Sequential([
    # pierwsza konwulucja
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    # druga konwulucja
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2), 
    # trzecia konwolucja
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # czwarta konwolucja
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # splaszczenie wynikow, w celu przekazania do warstwy gestej
    tf.keras.layers.Flatten(),
    # warstwa gesta 512 nerownow
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [22]:
history = model.fit(training_generator, epochs=10, validation_data=validation_generator, verbose=1)

Epoch 1/10
79/79 [==============================] - 31s 384ms/step - loss: 2.1362 - accuracy: 0.3637 - val_loss: 1.0262 - val_accuracy: 0.5860
Epoch 2/10
79/79 [==============================] - 29s 369ms/step - loss: 0.9099 - accuracy: 0.5823 - val_loss: 0.2841 - val_accuracy: 0.9758
Epoch 3/10
79/79 [==============================] - 30s 379ms/step - loss: 0.5293 - accuracy: 0.7798 - val_loss: 0.0858 - val_accuracy: 0.9973
Epoch 4/10
79/79 [==============================] - 30s 377ms/step - loss: 0.3406 - accuracy: 0.8801 - val_loss: 0.1484 - val_accuracy: 0.9597
Epoch 5/10
79/79 [==============================] - 30s 382ms/step - loss: 0.2483 - accuracy: 0.9140 - val_loss: 0.1521 - val_accuracy: 0.9462
Epoch 6/10
79/79 [==============================] - 31s 392ms/step - loss: 0.1845 - accuracy: 0.9401 - val_loss: 0.0437 - val_accuracy: 0.9892
Epoch 7/10
79/79 [==============================] - 31s 384ms/step - loss: 0.1323 - accuracy: 0.9497 - val_loss: 1.2700 - val_accuracy: 0.6478

In [24]:
file_name = './tmp/test-data/'
zip_ref = zipfile.ZipFile('./rps-validation.zip', 'r')
zip_ref.extractall(file_name)
zip_ref.close()

In [103]:
# testujemy model na danych ktorych nie widzial

directory =  './tmp/test-data/'

for file in os.listdir(directory):
    if file.endswith('png'):
        path = './tmp/test-data/' + file
        
        img = image.load_img(path, target_size=(150, 150))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        
        images = np.vstack([x])
        classes = model.predict(images, batch_size=10)
        print(file)
        print(classes[0])
        

# Wyniki: klasy sa wczytane w kolejnosci alfabetycznej(ang) (paper, rock, scissors) co odpoiwada [1, 0, 0 ] - paper,
# a w przypadku - [0, 1, 0] - rock 

paper8.png
[1. 0. 0.]
rock7.png
[0. 1. 0.]
scissors6.png
[0. 0. 1.]
paper9.png
[1. 0. 0.]
scissors-hires1.png
[0. 0. 1.]
paper-hires1.png
[1. 0. 0.]
rock3.png
[0. 1. 0.]
rock-hires1.png
[0. 1. 0.]
paper4.png
[1. 0. 0.]
paper-hires2.png
[1. 0. 0.]
scissors3.png
[0. 0. 1.]
scissors5.png
[0. 0. 1.]
scissors8.png
[0. 0. 1.]
rock1.png
[0. 1. 0.]
paper5.png
[1. 0. 0.]
paper1.png
[1. 0. 0.]
paper6.png
[1. 0. 0.]
rock-hires2.png
[0. 1. 0.]
scissors-hires2.png
[0. 0. 1.]
scissors1.png
[0. 0. 1.]
rock9.png
[0. 1. 0.]
rock8.png
[0. 1. 0.]
scissors7.png
[0. 0. 1.]
rock6.png
[0. 1. 0.]
rock4.png
[0. 1. 0.]
paper2.png
[1. 0. 0.]
scissors9.png
[0. 0. 1.]
rock5.png
[0. 1. 0.]
scissors4.png
[0. 0. 1.]
paper7.png
[1. 0. 0.]
scissors2.png
[0. 0. 1.]
rock2.png
[0. 1. 0.]
paper3.png
[1. 0. 0.]


In [99]:
classes[0][1] == np.float32(1)

False